# Feature Engineering

### 1. Install Dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

import warnings
warnings.filterwarnings("ignore")

### 2. Load Data

In [29]:
df = pd.read_csv('../data/processed/TelcoCustomerChurnPredictionDataCleaned.csv')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### 4. Tenure categories

In [30]:
print(f"Min tenure in data set: {df['tenure'].min()}")
print(f"Max tenure in data set: {df['tenure'].max()}")


Min tenure in data set: 0
Max tenure in data set: 72


In [31]:
df["tenure_binns"] = pd.cut(
                                df["tenure"], 
                                bins=3,
                                labels=["New", "Established", "Loyal"],
                                include_lowest=True
                            )
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_binns
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,New
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,Established
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,New
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Established
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,New


### 4. Service adoption score

- It’s a composite feature that measures how many services a customer is subscribed to.
    - Customers with higher adoption might be stickier (less churn).
    - Customers with low adoption might be at higher churn risk.

In [32]:
service_features = [
                        "PhoneService", "InternetService", "OnlineSecurity", "DeviceProtection",
                        "OnlineBackup", "TechSupport",	"StreamingTV",	"StreamingMovies"
                   ]

df_services = df[service_features].copy()

df_services["InternetService"] = df_services["InternetService"].apply(lambda x : 0 if x == "No" else 1)

df_services[service_features] = df_services[service_features].replace({"Yes" : 1, "No" : 0, "No phone service": 0, "No internet service": 0})

df["ServiceAdoptionScore"] = df_services.sum(axis=1)

df.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_binns,ServiceAdoptionScore
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,New,2
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,No,One year,No,Mailed check,56.95,1889.50,No,Established,4
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,New,4
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Established,4
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,New,2


### 5. Average monthly charges per service

AvgMonthlyChargesPerService = MonthlyCharges / ServiceAdoptionScore 

In [33]:
df["AvgMonthlyChargesPerService"] = np.where(
                                                df["ServiceAdoptionScore"] > 0,
                                                df["MonthlyCharges"] / df["ServiceAdoptionScore"],
                                                0
                                            )
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_binns,ServiceAdoptionScore,AvgMonthlyChargesPerService
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,New,2,14.9250
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,One year,No,Mailed check,56.95,1889.50,No,Established,4,14.2375
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,New,4,13.4625
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Established,4,10.5750
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,New,2,35.3500


### 6. Payment reliability indicators

In [34]:
df["PaymentMethod"].unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

'Electronic check' ->  Low
'Mailed check' -> Low
'Bank transfer (automatic)' -> Medium
'Credit card (automatic)' -> High

In [42]:
# Payement Risk Indicator
payment_risk_map = {
    "Electronic check": 1,   # high risk
    "Mailed check": 0.7,     # medium risk
    "Bank transfer (automatic)": 0.3,
    "Credit card (automatic)": 0.2  # lowest risk
}

df["PaymentMethodRisk"] = df["PaymentMethod"].map(payment_risk_map)

# Payemetn relibility
df["ExpectedTotalCharges"] = df["tenure"] * df["MonthlyCharges"]

df["PayementReliabilityRatio"] = df["TotalCharges"] / df["ExpectedTotalCharges"]
df['isPayementReliable'] = (df["PayementReliabilityRatio"] > 0.9).astype(int) 

# df = df.drop("ExpectedTotalCharges", "PayementReliabilityRatio")


In [43]:
df.to_csv('../data/processed/TelcoCustomerChurnPredictionFeatureEngineered.csv', index=False)